# Regression with TensorFlow on Alvis
This will introduce the very basics of using TensorFlow on Alvis.

As usual with Python the first step will be to load the necessary packages.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Constructing the data
In this step we generate a very simple dataset.

In [ ]:
def f_true(x, slope=0.5, bias=0.3):
    '''The true underlying relation.'''
    return slope * x + bias

def get_data(n_points, noise_level=0.1, true_function=f_true, **tf_kwargs):
    '''Generates noisy data from true_function.
    Arguments:
        n_points (int): Number of datapoints to generate
        noise_level (float): Std of gaussian noise to be added
        true_function (callable): The noiseless underlying function
        **function_kwargs: Optional key-word arguments passed to true_function
    '''
    x = tf.random.uniform((n_points, 1), minval=-1, maxval=1)
    y = true_function(x, **tf_kwargs) + tf.random.normal((n_points, 1), stddev=noise_level)
    return x, y

In [ ]:
x, y = get_data(300)

### Take a look at the data
As this is a notebook we can use the fact that we can easily take a look at graphical objects.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
plt.plot(x, y, '.', label="Data")
x_plot = tf.linspace(-1, 1, 20)
plt.plot(x_plot, f_true(x_plot), label="Noiseless relation")
plt.xlabel("X")
plt.ylabel("Y")

plt.legend();

## Constructing the model

In TensorFlow 2.0 the two main ways to construct models is either through the Sequential API or the Functional API. The sequential API catches the most common use cases were all of the layers in your deep learning are handled sequentially, the functional API on the other hand can handle computational graphs that are a lot more complex.

In this example we will use the functional API even if the Sequential API would have been sufficient.

In [ ]:
inputs = keras.Input(shape=(1,))
linear = layers.Dense(1)
outputs = linear(inputs)

# Instantiate the model
model = keras.Model(inputs=inputs, outputs=outputs, name="linear_model")
model.summary()


Note that in this simple case, we could have simply done
```python
model = keras.Sequential(layers.Dense(1, input_shape=(1,)))
```
directly, but we will build on this simple model later.


## Training the model
Here we will use gradient descent to train our regression model on the data we have generated.

In [ ]:
model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.SGD(learning_rate=0.1, momentum=0.3),
)

history = model.fit(x, y, epochs=20, batch_size=x.shape[0])


## Evaluating the model
Here we generate new data that we can use to evaluate the model performance.

In [ ]:
test_loss = model.evaluate(*get_data(100))

print(f"Test loss: {test_loss:.4g}")

As a side note, if you are doing computations with tensors that you do not want to update during training you can use
```python
out_tensor = torch.stop_gradient(my_func(in_tensor))
```
or simply specify that they do not require gradients directly
```python
# For specific tensor
my_constant = tf.constant(value)
```

Compared to PyTorch `model(inputs)` does not by default compute the gradients as the backpropagation is handled by the fit method. If you want to handle this yourself you can check out [tensorflow.GradientTape](https://www.tensorflow.org/guide/autodiff).

### Visualising model predictions

In [ ]:
plt.figure()
plt.plot(x, y, '.', label="Data")
x_plot = tf.linspace(-1, 1, 20)
plt.plot(x_plot, f_true(x_plot), label="Noiseless relation")
plt.xlabel("X")
plt.ylabel("Y")

# Add model prediction
plt.plot(x_plot, model.predict(x_plot), label="Predicted relation")

plt.legend();


## Your own model
In TensorFlow 2 most use cases can be handled by the [Sequential](https://www.tensorflow.org/guide/keras/sequential_model) or [Functional API](https://www.tensorflow.org/guide/keras/functional). This can then be extended by writing your own [custom layers](https://www.tensorflow.org/tutorials/customization/custom_layers), [custom losses](https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough) or [custom metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Metric).

### Excercises (optional)
1. Implement your own linear regression model with a fixed bias of 0.3, this can be done in several
different ways. Depending on your approach you might want to take a look at creating your own custom layer or use go very low level and you can subclass `tf.keras.Model` and use `tf.Variable` and `tf.constant`.

2. Train your model.

In [ ]:
# Your code here
my_model = None


In [ ]:
def check_model(model):
    if model is None:
        raise ValueError("Model is None.")
    x, y = get_data(1000)
    try:
        model(x)
    except Exception as e:
        print("Your model doesn't seem to handle input tensors of shape", x.size())
        raise e

    # Check bias
    bias_failure = False
    def check_bias(model):
        model_bias = model(tf.zeros((1, 1)))
        try:
            tf.debugging.assert_near(model_bias, tf.constant([[0.3]]))
        except tf.errors.InvalidArgumentError as e:
            nonlocal bias_failure
            bias_failure = True
            print(model.weights)
            print(f"Failure: The bias is {float(model_bias)}, not 0.3")
    check_bias(model)
    
    # Check that training changes the model performance
    if len(model.weights) != 1:
        print("Failure: Your model doesn't seem to only have a slope parameter.")
    
    # Check bias after training
    model_copy = tf.keras.models.clone_model(model)
    model_copy.set_weights([(var + 1) * 10 for var in model.weights])

    if not bias_failure:
        print("Checking model bias after changing variable values...")
        check_bias(model_copy)

In [ ]:
check_model(my_model)

## Running on a single GPU
For this example you will need access to a GPU, on Alvis there are four T4 GPUs
available on the login node, to see their status you can use the command
`nvidia-smi`. If they seem to be available then you can go ahead and use one of
them for the following excercises, otherwise you will have to submit a job.

You can use the Alvis OnDemand portal or submit a job manually with sbatch.

If you are going to submit a job you can modify the `jobscript-tensorflow.sh`
file, if you have forgotten what to think about when constructing a job script
you can take a look at part 1 and/or the introduction slides.

Now for the actual coding. In TensorFlow GPU acceleration on a single GPU is
handled transparently with no code changes, an overview of this you can find
in the TF [GPU Guide](https://www.tensorflow.org/guide/gpu).

To make sure that your code runs on a GPU you can first check
```python
tf.config.list_physical_devices('GPU')
```
and for detailed information on all operations use
```python
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
```
note that you'll have to do this at the very start of your code.

If you want to run on a specific device you can use
```
with tf.device("/GPU:0"):
    # Operations on GPU 0
```


### Excercises
1. Use `nvidia-smi` to find out about current GPU usage

In [ ]:
%%bash
nvidia-smi


2. Decide if you will do the following excercises on the log-in node or if you
will submit a job. Or both for the experience
3. Modify the below code to see if your training runs on GPU

In [ ]:
# You should modify this block so that it runs on a GPU
# and later also change the amount of data to train on

x, y = get_data(300)

model = tf.keras.Sequential(
    layers.Dense(1),
)
model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.SGD(learning_rate=0.1, momentum=0.3),
)

model.fit(x, y);

4. When you think you've succeded submit it with the jobscript.sh
5. Redo the GPU training but now do it with 1 billion data points. 
6. Use `sacct` in a terminal to find the job ID and then run `job_stats.py JOB_ID`
after substituting in the job ID. Look at the generated link. Are you using the GPU well?